# Script for predictions 

Here, we assume that there is a saved model which we want to use for predictions, and use it on a target image.

In [1]:
# Imports
import os, keras, re, pathlib, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from src import helper_models, helper_data
from PIL import Image

Using TensorFlow backend.


In [2]:
# Paths
modelpath='./out/conv2/2020-01-19_22h44m18s/'
modelpath='./out/conv2/2020-01-21_17h11m33s/'

argsname=modelpath+'args.txt'

# Test on Daphnia
target='daphnia'
im_names=['./data/zooplankton_trainingset_15oct/'+target+'/'+name for name in os.listdir('./data/zooplankton_trainingset_15oct/'+target+'/') if '.jpeg' in name]
print('We will predict the class of {} images belonging to the {} class'.format(len(im_names),target))

We will predict the class of 535 images belonging to the daphnia class


In [3]:
# Read Arguments
with open(argsname,'r') as fargs:
    args=fargs.read()
print(args)
layers=[None,None]
for s in re.split('[\,,\),\(]',args):
    if 'height' in s:
        height=np.int64(re.search('(\d+)',s).group(1))
    if 'width' in s:
        width=np.int64(re.search('(\d+)',s).group(1))
    if 'depth' in s:
        depth=np.int64(re.search('(\d+)',s).group(1))
    if 'model' in s:
        modelname=re.search('=\'(.+)\'$',s).group(1)
    if 'resize' in s:
        resize=re.search('=\'(.+)\'$',s).group(1)
    if 'layers' in s: #first layer
        layers[0]=np.int64(re.search('=\[(.+)$',s).group(1))
    if re.match('^ \d+',s): #second layer
        layers[1]=np.int64(re.match('^ (\d+)',s).group(1))
    if 'datapath' in s:
        datapath=re.search('=\'(.+)\'$',s).group(1)
    if 'outpath' in s:
        outpath=re.search('=\'(.+)\'$',s).group(1)
    
classes = [ name for name in os.listdir(datapath) if os.path.isdir(os.path.join(datapath, name)) ]

# Double check that classes are correct
classes_dict=np.load(modelpath+'classes.npy',allow_pickle=True).item()


Namespace(aug=True, bs=8, datapath='./data/zooplankton_trainingset_15oct/', depth=3, height=128, layers=[256, 128], load=None, lr=0.0001, model='conv2', opt='sgd', outpath='./out/', plot=False, resize='keep_proportions', testSplit=0.2, totEpochs=300, verbose=False, width=128)
Training-time: 10629.327492952347 seconds



In [5]:
from stat import S_ISREG, ST_CTIME, ST_MODE
# Choose model. We load the latest created .hdf5 file, since later is better
entries = [modelpath+'/'+entry for entry in os.listdir(modelpath) if '.hdf5' in entry]
entries = ((os.stat(path), path) for path in entries)
entries = ((stat[ST_CTIME], path) for stat, path in entries if S_ISREG(stat[ST_MODE]))
modelfile=sorted(entries)[-1][1]

# Initialize and load model
print('Model:',modelname)
model=keras.models.load_model(modelfile)

# model = keras.models.load_model(modelfile)
# opt = keras.optimizers.SGD(lr=0.001, nesterov=True)
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print('Loaded ',modelfile)

Model: conv2
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Loaded  ./out/conv2/2020-01-21_17h11m33s//bestweights.hdf5


In [6]:
def load_images(im_names, width, height, depth, modelname, resize):
    ''' 
    Function that loads a list of images given in im_names, and returns 
    them in a numpy format that can be used by the classifier
    '''
    npimages=np.ndarray((len(im_names),width,height,depth)) if modelname != 'mlp' else np.ndarray((len(im_names),width*height*depth))

    for i,im_name in enumerate(im_names):
        image = Image.open(im_name)
        if resize == 'acazzo':
            image = image.resize((width,height))
        else:
            # Set image's largest dimension to target size, and fill the rest with black pixels
            image,rescaled = helper_data.ResizeWithProportions(image, width) # width and height are assumed to be the same (assertion at the beginning)
            npimage=np.array(image.copy())
        if model == 'mlp':
            npimage = npimage.flatten()
        npimages[i]=npimage
        if len(im_names)==1:
            image.show()
        image.close()    
    return npimages/255.0


In [7]:
# Load images
npimages=load_images(im_names, width, height, depth, modelname, resize)


In [9]:
# Print prediction
predictions=model.predict(npimages).argmax(axis=1)
count=0
for i in range(len(npimages)):
    if classes_dict['name'][predictions[i]] == 'daphnia':
        count+=1
print('Accuracy: ', count/len(npimages))


Accuracy:  0.8411214953271028
